In [1]:
# Import packages

import pandas as pd
import math
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import pyart
import glob
import matplotlib.patheffects as path_effects
from matplotlib.colors import BoundaryNorm, LinearSegmentedColormap
from pyproj import Transformer
import radlib
import os
import h5py
import matplotlib.dates as mdates
from scipy.ndimage import gaussian_filter1d
from cartopy.feature import NaturalEarthFeature
import xmltodict, geopandas, geojson, xml #xml and json do not exist
from datetime import datetime, timedelta, timezone
import geopy.distance
from datetime import datetime, timedelta
import numpy.matlib as npm
import copy
from scipy.signal import convolve2d
from astropy.convolution import convolve
import scipy.ndimage as ndi
import re
from skimage.draw import polygon

from pprint import pprint
from pysteps import io, motion, rcparams
from pysteps.utils import conversion, transformation
from pysteps.visualization import plot_precip_field, quiver

import json

from mpl_toolkits.axes_grid1.anchored_artists import AnchoredSizeBar
import matplotlib.font_manager as fm

import polars as pl

import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling

import geopandas as gpd
from shapely.geometry import Point

from mpl_toolkits.mplot3d import Axes3D

from shapely.geometry import Polygon


from geopandas import GeoDataFrame
import datetime

import matplotlib.patches as patches

import geojson as gs


os.environ["library_metranet_path"] = "/store_new/mch/msrad/idl/lib/radlib4/" # needed for pyradlib
os.environ["METRANETLIB_PATH"] = "/store_new/mch/msrad/idl/lib/radlib4/" # needed for pyart_mch


# Data Imports, processing, norming, etc.


# GPSConverter for coordinate transformation
class GPSConverter(object):
    '''
    GPS Converter class which is able to perform convertions between the 
    CH1903 and WGS84 system.
    '''
    # Convert CH y/x/h to WGS height
    def CHtoWGSheight(self, y, x, h):
        # Axiliary values (% Bern)
        y_aux = (y - 600000) / 1000000
        x_aux = (x - 200000) / 1000000
        h = (h + 49.55) - (12.60 * y_aux) - (22.64 * x_aux)
        return h

    # Convert CH y/x to WGS lat
    def CHtoWGSlat(self, y, x):
        # Axiliary values (% Bern)
        y_aux = (y - 600000) / 1000000
        x_aux = (x - 200000) / 1000000
        lat = (16.9023892 + (3.238272 * x_aux)) + \
                - (0.270978 * pow(y_aux, 2)) + \
                - (0.002528 * pow(x_aux, 2)) + \
                - (0.0447 * pow(y_aux, 2) * x_aux) + \
                - (0.0140 * pow(x_aux, 3))
        # Unit 10000" to 1" and convert seconds to degrees (dec)
        lat = (lat * 100) / 36
        return lat

    # Convert CH y/x to WGS long
    def CHtoWGSlng(self, y, x):
        # Axiliary values (% Bern)
        y_aux = (y - 600000) / 1000000
        x_aux = (x - 200000) / 1000000
        lng = (2.6779094 + (4.728982 * y_aux) + \
                + (0.791484 * y_aux * x_aux) + \
                + (0.1306 * y_aux * pow(x_aux, 2))) + \
                - (0.0436 * pow(y_aux, 3))
        # Unit 10000" to 1" and convert seconds to degrees (dec)
        lng = (lng * 100) / 36
        return lng

    # Convert decimal angle (° dec) to sexagesimal angle (dd.mmss,ss)
    def DecToSexAngle(self, dec):
        degree = int(math.floor(dec))
        minute = int(math.floor((dec - degree) * 60))
        second = (((dec - degree) * 60) - minute) * 60
        return degree + (float(minute) / 100) + (second / 10000)
		
    # Convert sexagesimal angle (dd.mmss,ss) to seconds
    def SexAngleToSeconds(self, dms):
        degree = 0 
        minute = 0 
        second = 0
        degree = math.floor(dms)
        minute = math.floor((dms - degree) * 100)
        second = (((dms - degree) * 100) - minute) * 100
        return second + (minute * 60) + (degree * 3600)

    # Convert sexagesimal angle (dd.mmss) to decimal angle (degrees)
    def SexToDecAngle(self, dms):
        degree = 0
        minute = 0
        second = 0
        degree = math.floor(dms)
        minute = math.floor((dms - degree) * 100)
        second = (((dms - degree) * 100) - minute) * 100
        return degree + (minute / 60) + (second / 3600)
    
    # Convert WGS lat/long (° dec) and height to CH h
    def WGStoCHh(self, lat, lng, h):
        lat = self.DecToSexAngle(lat)
        lng = self.DecToSexAngle(lng)
        lat = self.SexAngleToSeconds(lat)
        lng = self.SexAngleToSeconds(lng)
        # Axiliary values (% Bern)
        lat_aux = (lat - 169028.66) / 10000
        lng_aux = (lng - 26782.5) / 10000
        h = (h - 49.55) + (2.73 * lng_aux) + (6.94 * lat_aux)
        return h

    # Convert WGS lat/long (° dec) to CH x
    def WGStoCHx(self, lat, lng):
        lat = self.DecToSexAngle(lat)
        lng = self.DecToSexAngle(lng)
        lat = self.SexAngleToSeconds(lat)
        lng = self.SexAngleToSeconds(lng)
        # Axiliary values (% Bern)
        lat_aux = (lat - 169028.66) / 10000
        lng_aux = (lng - 26782.5) / 10000
        x = ((200147.07 + (308807.95 * lat_aux) + \
            + (3745.25 * pow(lng_aux, 2)) + \
            + (76.63 * pow(lat_aux,2))) + \
            - (194.56 * pow(lng_aux, 2) * lat_aux)) + \
            + (119.79 * pow(lat_aux, 3))
        return x

	# Convert WGS lat/long (° dec) to CH y
    def WGStoCHy(self, lat, lng):
        lat = self.DecToSexAngle(lat)
        lng = self.DecToSexAngle(lng)
        lat = self.SexAngleToSeconds(lat)
        lng = self.SexAngleToSeconds(lng)
        # Axiliary values (% Bern)
        lat_aux = (lat - 169028.66) / 10000
        lng_aux = (lng - 26782.5) / 10000
        y = (600072.37 + (211455.93 * lng_aux)) + \
            - (10938.51 * lng_aux * lat_aux) + \
            - (0.36 * lng_aux * pow(lat_aux, 2)) + \
            - (44.54 * pow(lng_aux, 3))
        return y

    def LV03toWGS84(self, east, north, height):
        '''
        Convert LV03 to WGS84 Return a array of double that contain lat, long,
        and height
        '''
        d = []
        d.append(self.CHtoWGSlat(east, north))
        d.append(self.CHtoWGSlng(east, north))
        d.append(self.CHtoWGSheight(east, north, height))
        return d
        
    def WGS84toLV03(self, latitude, longitude, ellHeight):
        '''
        Convert WGS84 to LV03 Return an array of double that contaign east,
        north, and height
        '''
        d = []
        d.append(self.WGStoCHy(latitude, longitude))
        d.append(self.WGStoCHx(latitude, longitude))
        d.append(self.WGStoCHh(latitude, longitude, ellHeight))
        return d

# c_transform
def c_transform(lon,lat):
    """
    transforms arrays of lat/lon to chx/chy

    Parameters
    ----------
    lon : float
        longitude.
    lat : float
        latitude.

    Returns
    -------
    chx : float
        chx in m.
    chy : float
        chy in m.

    """
    converter = GPSConverter()
    chx=np.zeros([len(lon)])
    chy=np.zeros([len(lon)])
    for n in range(len(lon)):
        chx[n],chy[n],z=converter.WGS84toLV03(lat[n], lon[n], 0)
    return chx,chy

# transform_c 
def transform_c(chx,chy):
    """
    transforms arrays of chx/chy to lat/lon
    
    Parameters
    -------
    chx : float
        chx in m.
    chy : float
        chy in m.
        
    Returns
    ----------
    lon : float
        longitude.
    lat : float
        latitude.



    """
    converter = GPSConverter()
    lon=np.zeros([len(chx)])
    lat=np.zeros([len(chy)])
    for n in range(len(lon)):
        lat[n], lon[n],z=converter.LV03toWGS84(chx[n],chy[n], 0)
    return lon,lat



## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119

Pysteps configuration file found at: /scratch/mch/fackerma/miniforge3/envs/testenv/lib/python3.12/site-packages/pysteps/pystepsrc



In [80]:


# Define TRT reading function
def read_TRT(path, file=0, ttime=0):
    """
    Read .trt or .json file containing TRT output
    Returns dataframe with attributes and gridded TRT cells

    Parameters
    ----------

    path : string
        path, where to look for files.
    file: string
        filename
    ttime : string
        timestep to find files for.
    Requires either filename or timestep
   
    Returns
    -------
    trt_df : dataframe
        TRT cells and attributes of the timestep.
    cells: list
        Gridded TRT cells per timestep
    timelist: list
        timesteps

    """
   
    o_x=255000
    o_y=-160000
    lx=710; ly=640
    cells=np.zeros([ly,lx])
    if file == 0:
        file=glob.glob(path["lomdata"]+'*'+ttime+'*json*')
        if len(file)>0: flag=1
        else:
            file=glob.glob(path["lomdata"]+'*'+ttime+'*'+'.trt')[0]
            flag=0
    else:
        if 'json' in file: flag=1; ttime=file[-20:-11]
        else: flag=0; ttime=file[-15:-6]
        file=[file]
   
    if flag==1:
        with open(file[0]) as f: gj = geojson.FeatureCollection(gs.load(f))
        trt_df=geopandas.GeoDataFrame.from_features(gj['features'])
        if len(trt_df)>0:
          # print(trt_df.lon.values.astype(float))
          chx, chy = c_transform(trt_df.lon.values.astype(float),trt_df.lat.values.astype(float))
          trt_df['chx']=chx.astype(str); trt_df['chy']=chy.astype(str)
          for n in range(len(trt_df)):
              lon,lat=trt_df.iloc[n].geometry.boundary.xy
              # print(trt_df.iloc[n])
              chx, chy = c_transform(lon,lat)
              # trt_df.iloc[n]['chx']=chx.astype(str); trt_df.iloc[n]['chy']=chy.astype(str)
              #transform.c_transform(trt_df.iloc[n].lon.values,trt_df.iloc[n].lat.values)
              ix=np.round((chx-o_x)/1000).astype(int)
              iy=np.round((chy-o_y)/1000).astype(int)
              rr, cc = polygon(iy, ix, cells.shape)
              # print(lat,lon,chx,chy,ix,iy)
              cells[rr,cc]=int(trt_df.traj_ID.iloc[n]);
        else: cells=[]
    else:
        data = pd.read_csv(file).iloc[8:]
        headers = pd.read_csv(file).iloc[7:8].iloc[0][0].split()
        trt_df = pd.DataFrame()
        geometries = []  # New list to store geometries

        for n in range(len(data)):
            # ... (existing code remains the same)
            t = data.iloc[n].str.split(';', expand=True)
            trt_df.loc[n, 'traj_ID'] = int(t[0].values)
            trt_df.loc[n, 'time'] = int(t[1].values)
            trt_df.loc[n, 'lon'] = t[2].values.astype(float)
            trt_df.loc[n, 'lat'] = t[3].values.astype(float)
            trt_df.loc[n, 'ell_L'] = t[4].values.astype(float)
            trt_df.loc[n, 'ell_S'] = t[5].values.astype(float)
            trt_df.loc[n, 'ell_or'] = t[6].values.astype(float)
            trt_df.loc[n, 'area'] = t[7].values.astype(float)
            trt_df.loc[n, 'vel_x'] = t[8].values.astype(float)
            trt_df.loc[n, 'vel_y'] = t[9].values.astype(float)
            trt_df.loc[n, 'det'] = t[10].values.astype(float)
            trt_df.loc[n, 'RANKr'] = t[11].values.astype(float)
            trt_df.loc[n, 'CG-'] = t[12].values.astype(float)
            trt_df.loc[n, 'CG+'] = t[13].values.astype(float)
            trt_df.loc[n, 'CG'] = t[14].values.astype(float)
            trt_df.loc[n, '%CG+'] = t[15].values.astype(float)
            trt_df.loc[n, 'ET45'] = t[16].values.astype(float)
            trt_df.loc[n, 'ET45m'] = t[17].values.astype(float)
            trt_df.loc[n, 'ET15'] = t[18].values.astype(float)
            trt_df.loc[n, 'ET15m'] = t[19].values.astype(float)
            trt_df.loc[n, 'VIL'] = t[20].values.astype(float)
            trt_df.loc[n, 'maxH'] = t[21].values.astype(float)
            trt_df.loc[n, 'maxHm'] = t[22].values.astype(float)
            trt_df.loc[n, 'POH'] = t[23].values.astype(float)
            trt_df.loc[n, 'MESHS'] = t[24].values.astype(float)
            trt_df.loc[n, 'Dvel_x'] = t[25].values.astype(float)
            trt_df.loc[n, 'Dvel_y'] = t[26].values.astype(float)
            chx, chy = c_transform([trt_df.loc[n, 'lon']], [trt_df.loc[n, 'lat']])
            ix = np.round((chx - o_x) / 1000).astype(int)
            if ix >= 710: ix = 709
            iy = np.round((chy - o_y) / 1000).astype(int)
            if iy >= 640: iy = 639
            n2 = 27
            #if int(ttime) >= 221520631: n2 = 82
            tt = np.array(t)[0, n2:-1]
            tt = np.reshape(tt, [int(len(tt) / 2), 2])
            trt_df.loc[n, 'chx'] = chx
            trt_df.loc[n, 'chy'] = chy
            lat = tt[:, 1].astype(float)
            lon = tt[:, 0].astype(float)
            chx, chy = c_transform(lon, lat)
            ix = np.round((chx - o_x) / 1000).astype(int)
            iy = np.round((chy - o_y) / 1000).astype(int)
            rr, cc = polygon(iy, ix, cells.shape)
            cells[rr, cc] = int(t[0].values)
            # Create polygon for this cell
            polygon_coords = list(zip(lon, lat))
            cell_polygon = Polygon(polygon_coords)
            geometries.append(cell_polygon)
        
        # Add geometry column to trt_df
        trt_df['geometry'] = geometries
    
        # Convert trt_df to GeoDataFrame
        trt_df = gpd.GeoDataFrame(trt_df, geometry='geometry', crs="EPSG:4326")

    timelist=[str(ttime)]
    return trt_df, [cells], timelist


In [81]:

valid_time3 = '1600'
valid_time4= valid_time3 + '0'
path_trtc = {"lomdata": "/scratch/mch/fackerma/orders/case_150619/TRTC/"}  # Replace with your actual path
#path_trtc = {"lomdata": "/scratch/mch/fackerma/orders/case_240823/TRTC/"}
trt_df, cells_list, timelist = read_TRT(path_trtc, ttime=valid_time4)
# Extract the first element from the list of cells
cells = cells_list[0]


/tmp/ipykernel_178738/2408269897.py:65: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  headers = pd.read_csv(file).iloc[7:8].iloc[0][0].split()
/tmp/ipykernel_178738/2408269897.py:72: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  trt_df.loc[n, 'traj_ID'] = int(t[0].values)
/tmp/ipykernel_178738/2408269897.py:73: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  trt_df.loc[n, 'time'] = int(t[1].values)
/tmp/ipykernel_178738/2408269897.py:116: DeprecationWarnin

In [82]:
trt_df

,traj_ID,time,lon,lat,ell_L,ell_S,ell_or,area,vel_x,vel_y,det,RANKr,CG-,CG+,CG,%CG+,ET45,ET45m,ET15,ET15m,VIL,maxH,maxHm,POH,MESHS,Dvel_x,Dvel_y,chx,chy,geometry
0,2.019062e+15,2.019062e+11,6.7450,46.7625,24.18,8.54,76.8,331.0,29.65,45.86,45.0,19.0,26.0,1.0,27.0,3.0,8.9,6.1,12.7,11.2,24.0,7.8,3.4,0.0,2.0,10.48,16.21,547011.518545,179270.429003,"POLYGON ((6.6714 46.8631, 6.6717 46.8451, 6.65..."
1,2.019062e+15,2.019062e+11,7.0650,46.5771,27.85,13.80,134.3,342.0,18.98,32.53,45.0,10.0,38.0,6.0,44.0,13.0,8.2,4.0,13.1,10.8,11.5,7.8,2.8,0.0,1.0,4.73,6.66,571358.352850,158493.714884,"POLYGON ((6.9611 46.7387, 6.9481 46.7296, 6.89..."
2,2.019062e+15,2.019062e+11,6.1604,46.5229,37.45,14.38,118.7,610.0,16.10,29.80,42.0,4.0,0.0,0.0,0.0,0.0,2.7,2.5,9.7,8.8,2.5,2.8,2.4,0.0,0.0,7.48,11.67,501921.129541,153200.575372,"POLYGON ((5.9675 46.7032, 5.9547 46.6941, 5.94..."
3,2.019062e+15,2.019062e+11,7.1670,46.2519,9.41,4.77,81.3,82.0,37.44,37.69,42.0,7.0,8.0,0.0,8.0,0.0,6.4,3.4,11.9,10.0,5.0,6.4,2.8,0.0,1.0,6.22,6.11,579051.552777,122310.753079,"POLYGON ((7.1077 46.2984, 7.0948 46.2894, 7.09..."
4,2.019062e+15,2.019062e+11,7.0812,45.9291,6.21,5.25,164.7,58.0,26.50,34.56,42.0,7.0,8.0,0.0,8.0,0.0,7.0,3.5,11.6,10.8,4.0,6.6,3.6,0.0,1.0,9.37,12.22,572270.954983,86453.255148,"POLYGON ((7.0579 45.9924, 7.0581 45.9654, 7.04..."
5,2.019062e+15,2.019062e+11,7.1375,45.8127,10.66,6.24,41.9,125.0,26.50,26.56,42.0,14.0,3.0,0.0,3.0,0.0,8.4,6.7,11.9,10.8,12.0,7.0,5.2,0.0,1.0,1.40,1.97,576588.530942,73494.884636,"POLYGON ((7.1874 45.9028, 7.1745 45.8938, 7.16..."
6,2.019062e+15,2.019062e+11,7.3554,45.2687,5.30,3.07,102.0,32.0,18.52,14.87,42.0,7.0,0.0,0.0,0.0,0.0,6.3,3.7,11.3,10.2,5.0,6.2,3.8,0.0,1.0,6.55,5.26,593463.056907,12973.097571,"POLYGON ((7.343 45.3094, 7.3049 45.2824, 7.292..."
7,2.019062e+15,2.019062e+11,7.2253,45.2029,11.26,6.42,76.6,150.0,18.52,14.87,42.0,23.0,4.0,0.0,4.0,0.0,9.9,6.9,12.9,11.7,33.5,7.8,4.4,0.0,2.0,1.98,0.89,583228.812631,5676.515604,"POLYGON ((7.2412 45.2823, 7.2285 45.2733, 7.21..."
8,2.019062e+15,2.019062e+11,6.7747,46.3506,5.91,2.74,114.6,28.0,-0.00,0.00,39.0,5.0,0.0,0.0,0.0,0.0,2.6,2.6,9.8,9.5,1.5,2.6,2.2,0.0,1.0,-0.00,0.00,548892.079008,133463.195051,"POLYGON ((6.7302 46.3867, 6.7303 46.3777, 6.71..."
9,2.019062e+15,2.019062e+11,6.7861,46.2353,9.64,3.72,99.9,66.0,-0.00,0.00,39.0,3.0,0.0,0.0,0.0,0.0,2.5,2.5,9.2,9.0,1.5,2.8,2.2,0.0,0.0,-0.00,0.00,549662.776929,120638.931874,"POLYGON ((6.7316 46.2787, 6.7187 46.2696, 6.70..."


In [ ]:
valid_time3 = '1515'
valid_time4= valid_time3 + '0'
#path_trtc = {"lomdata": "/scratch/mch/fackerma/orders/case_150619/TRTC/"}  # Replace with your actual path
path_trtc = {"lomdata": "/scratch/mch/fackerma/orders/case_240823/TRTC/"}
trt_df, cells_list, timelist = read_TRT(path_trtc, ttime=valid_time4)
# Extract the first element from the list of cells
cells = cells_list[0]


,geometry,traj_ID,yyyymmddHHMM,lon,lat,ell_L,ell_S,ell_or,area,vel_x,vel_y,det,RANKr,CG-,CG+,CG,%CG+,ET45,ET45m,ET15,ET15m,VIL,maxH,maxHm,POH,MESHS,Dvel_x,Dvel_y,VILD,POHm3,POH90,POH75,nrPOH,nrPOH90,nrPOH75,MESHSm3,MESHS90,MESHS75,nrMESHS,nrMESHS90,nrMESHS75,HZTm3,HZT90,HZT75,D_HZT,dBZmax,dBZm3,dBZ90,dBZ75,nrdBZ90,nrdBZ75,area45,area57,IC,L_TOT,RRm3,RR90,RR75,nrRR90,nrRR75,IR108max,IR108med,IR108min,nrMESHSthr20,nrMESHSthr25,nrMESHSthr30,nrMESHSthr35,nrMESHSthr40,nrMESHSthr45,nrMESHSthr50,nrMESHSthr55,nrMESHSthr60,nrPOHthr001,nrPOHthr010,nrPOHthr020,nrPOHthr030,nrPOHthr040,nrPOHthr050,nrPOHthr060,nrPOHthr070,nrPOHthr080,nrPOHthr090,nrPOHthr100,chx,chy
0,"POLYGON ((5.4292 46.758, 5.4296 46.749, 5.4168...",2023082415100012,202308241515,5.4193,46.7096,7.30,3.50,56.2,48,36.00,28.00,48,16,0,0,0,0,9.7,7.5,15.1,12.1,17.0,7.8,4.6,92,0.0,0.00,2.83,1.4,91,90,86,32,3,7,0.0,0.0,0.0,0,0,0,0,0,0,0,55.5,54.5,54.0,53.0,4,11,48,0,25,25,5.7,5.2,4.5,2,3,-999.9,-999.9,-999.9,0,0,0,0,0,0,0,0,0,32,31,30,26,26,25,22,17,13,6,0,445601.7875984461,175144.78662031607
1,"POLYGON ((5.184 46.6724, 5.1714 46.6632, 5.172...",2023082413450185,202308241515,5.2752,46.6134,14.82,5.01,88.3,99,40.31,32.11,48,11,0,0,0,0,7.7,4.5,13.7,12.5,12.0,5.2,2.2,61,0.0,2.88,1.28,0.9,0,61,61,2,0,0,0.0,0.0,0.0,0,0,0,0,0,0,0,54.0,53.0,52.5,51.0,9,24,99,0,23,23,23.9,20.7,12.5,2,1,-999.9,-999.9,-999.9,0,0,0,0,0,0,0,0,0,2,2,2,1,1,1,1,0,0,0,0,434291.4657228222,164748.6795055712
2,"POLYGON ((5.0297 46.6153, 5.0171 46.606, 5.017...",2023082415050013,202308241515,5.0777,46.5461,10.20,6.24,103.8,124,40.37,26.24,48,23,0,0,0,0,10.1,7.9,14.9,13.1,26.0,8.6,4.6,96,2.6,0.79,2.27,2.0,94,90,82,102,10,25,2.2,2.6,2.6,4,0,0,0,0,0,0,59.5,59.0,57.5,55.0,12,30,124,16,16,16,35.9,34.2,31.0,3,2,-999.9,-999.9,-999.9,4,2,0,0,0,0,0,0,0,102,100,93,84,74,69,63,60,37,12,0,418943.00700263487,157707.28375736476
3,"POLYGON ((5.2534 46.2508, 5.2408 46.2416, 5.21...",2023082415050050,202308241515,5.2586,46.2173,6.55,2.75,72.1,37,41.81,23.04,45,13,1,0,1,0,7.9,6.2,13.0,12.2,10.5,6.2,2.4,65,0.0,1.46,0.60,0.9,60,65,42,18,1,4,0.0,0.0,0.0,0,0,0,0,0,0,0,54.0,52.0,52.0,50.0,3,9,37,0,29,30,8.3,8.1,6.4,1,2,-999.9,-999.9,-999.9,0,0,0,0,0,0,0,0,0,18,12,9,8,5,4,4,0,0,0,0,431795.6579070317,120768.46117418379
4,"POLYGON ((5.139 46.1945, 5.1265 46.1853, 5.113...",2023082415150046,202308241515,5.1586,46.1596,5.93,3.39,72.2,43,39.82,14.33,45,10,0,0,0,0,8.3,3.6,13.7,12.4,10.5,5.4,1.8,73,0.0,14.08,5.07,0.9,0,73,73,1,0,0,0.0,0.0,0.0,0,0,0,0,0,0,0,53.5,53.0,52.5,50.5,4,10,43,0,7,7,14.7,14.3,14.3,9,9,-999.9,-999.9,-999.9,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,0,0,0,423895.56304030464,114576.49247750622
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,"POLYGON ((7.167 44.8145, 7.1544 44.8055, 7.154...",2023082415050059,202308241515,7.2023,44.7728,6.18,2.02,131.2,23,20.65,-1.19,36,5,0,0,0,0,3.9,3.9,12.4,8.1,2.5,5.2,3.2,0,0.0,9.93,4.37,0.0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0,0,0,0,0,46.5,44.5,44.5,41.0,2,5,1,0,0,0,21.3,21.3,19.5,2,2,-999.9,-999.9,-999.9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,581276.2521249052,-42143.80660024976
61,"POLYGON ((7.0913 44.8053, 7.0914 44.7873, 7.07...",2023082412500094,202308241515,7.1419,44.7287,16.78,4.26,113.0,117,14.39,0.87,36,14,0,0,0,0,5.7,5.2,13.0,8.2,10.0,8.4,3.4,0,0.0,5.79,5.10,0.0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0,0,0,0,0,59.0,58.0,54.5,46.5,11,29,37,4,0,0,108.6,103.1,63.7,4,5,-999.9,-999.9,-999.9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,576473.1482830531,-47031.73887368428
62,"POLYGON ((9.1862 44.738, 9.1848 44.6931, 9.21 ...",2023082414250170,202308241515,9.2475,44.7038,7.11,2.37,106.0,36,7.16,0.10,36,7,0,0,0,0,3.9,3.4,10.5,10.1,4.0,4.0,2.6,0,0.0,1.45,0.92,0.3,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0,0,0,0,0,51.0,50.5,50.0,47.5,3,8,1

In [45]:
print(trt_df.dtypes.to_dict())



{'geometry': <geopandas.array.GeometryDtype object at 0x14efafda7680>, 'traj_ID': dtype('O'), 'yyyymmddHHMM': dtype('O'), 'lon': dtype('O'), 'lat': dtype('O'), 'ell_L': dtype('O'), 'ell_S': dtype('O'), 'ell_or': dtype('O'), 'area': dtype('O'), 'vel_x': dtype('O'), 'vel_y': dtype('O'), 'det': dtype('O'), 'RANKr': dtype('O'), 'CG-': dtype('O'), 'CG+': dtype('O'), 'CG': dtype('O'), '%CG+': dtype('O'), 'ET45': dtype('O'), 'ET45m': dtype('O'), 'ET15': dtype('O'), 'ET15m': dtype('O'), 'VIL': dtype('O'), 'maxH': dtype('O'), 'maxHm': dtype('O'), 'POH': dtype('O'), 'MESHS': dtype('O'), 'Dvel_x': dtype('O'), 'Dvel_y': dtype('O'), 'VILD': dtype('O'), 'POHm3': dtype('O'), 'POH90': dtype('O'), 'POH75': dtype('O'), 'nrPOH': dtype('O'), 'nrPOH90': dtype('O'), 'nrPOH75': dtype('O'), 'MESHSm3': dtype('O'), 'MESHS90': dtype('O'), 'MESHS75': dtype('O'), 'nrMESHS': dtype('O'), 'nrMESHS90': dtype('O'), 'nrMESHS75': dtype('O'), 'HZTm3': dtype('O'), 'HZT90': dtype('O'), 'HZT75': dtype('O'), 'D_HZT': dtype('O

In [48]:
trt_df['IR108min'].head

<bound method NDFrame.head of 0     -999.9
1     -999.9
2     -999.9
3     -999.9
4     -999.9
       ...  
60    -999.9
61    -999.9
62    -999.9
63    -999.9
64    -999.9
Name: IR108min, Length: 65, dtype: object>